# Developing with AsyncIO, design patterns

While heavy working with web APIs, databases, disc IO (espessialy network disks), performance issues startig to be more sensitive. Since few years AsyncIO is getting increasingly more popular. It switches context on I/O bound tasks in the single thread. It means that your process will be doing another things while waiting for I/O, thus improves performance of IO bound tasks. 
Because python has Global Interpreter Lock (GIL), code is executed one single thread at a time. So, theoreticaly AsyncIO is at least as fast as multythreading.  
  
I was wondering about asyncio design choices. Here is a short tutorial on 3 paterns that i have found. ( Thanks Mycola Novik)

Disclaimer: Performance of different tasks with theese patterns will change.

Execution of the sync code is separate thread pool or process pool. 
- no way to kill a thread if task hangs up
- C extentions can consume a lot of memory, some versions of Docker can behave unpredictable
- Convenient 

ProcessPoolExecutors works as well, but pickling 
- Easy to terminate
- Consumes a lot of memory 
- Create processeses asap - due to FORK method of creating processes ;)

In [ ]:
import asyncio 
loop = asyncio.get_event_loop()
executor = ThreadPoolExecutor(max_workers = 4)
async def test_ex():
    string="conn"
    in_exec = await loop.run_in_executor(executor, function, arguments)
    
loop.run_untill_complete(test_ex())

Execution of the async code in separate thread? ( inside sync code)

Usecases:   
- scrapping, 
- loading files/network io 

using (small) coroutines inside sync code == simetimes slows down the code!   be careful!   

In [ ]:
import aiohttp

async def fetch( session, url, loop):
    async with session.get(url) as response :
        data= await response.text()
        
def collect data(url_list):
    loop=asyncio.get_event_loop()
    
    session = aiohttp.get_event_loop()
    coros=[fetch(session, u, loop)] for u in url list]
    
    data = loop.run_untill_complete(asyncio.gather(*coros, loop=loop))
    loop.run_untill_complete(session.close())
    loop.close()
    return data

def main():
    url_list= []
    data=collect_data(url_list)
    process(data)


Application may delegate IO heavy tasks to dedicated lop in separate thread

In [ ]:
import asyncio, functools
from threading import Thread, Event

class AioThread(Thread):
    def __init__(self,*args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loop=None
        self.event=Event()
        
    def run(self):
        self.loop = asyncio.new_event_loop()
        asyncio.set_event_loop(self._loop)
        self.loop.call_soon(self.event.set)
        self.loop.run_forever()
        
    def add_task(self, coro):
        fut=asyncio.run_couroutine_threadsafe(core, loop=self.loop)
        return fut
    
    def finalize(self, timeout=None):
        self._loop.call_soon_threadsafe(self._loop.stop)
        self.join(timeout=timeout)
        

!!!  ? check if loop have been started  
finalize thread properly 

In [ ]:
def main():
    aiothread=AioThread()
    aiothread.start()
    aiothread.event.wait()
    
    loop=aiothread.loop
    coro=asyncio.sleep(1,loop=loop)
    future= aiothread.add_task(coro)
    try:
        result = future.result(timeout)
    except asyncio.TimeoutError:
        print ('Coro is too long, cancelling')
        future.cancel()   # run in the main thread --- 
    except Exeption as exc:
        print ( "Coro rised exeption", exc)